In [12]:
from transformers import AutoImageProcessor, ResNetModel
import torch
from datasets import load_dataset

In [38]:
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

print("Image shape: ", image)

image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetModel.from_pretrained("microsoft/resnet-50")

inputs = image_processor(image, return_tensors="pt")

print("Inputs shape :", inputs.pixel_values.shape)

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs
list(last_hidden_states)
print(last_hidden_states.shape)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Inputs shape : torch.Size([1, 3, 224, 224])
Features to extract: ['stem']
Output shape from 'stem': torch.Size([1, 2048, 7, 7])


In [11]:

image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetModel.from_pretrained("microsoft/resnet-50")

inputs = image_processor(image, return_tensors="pt")
# Function to register a hook
outputs = {}
def get_features(name):
    def hook(model, input, output):
        outputs[name] = output.detach()
    return hook

# Register hook to the desired layers
#print(model.modules)

model.

model.layer1.register_forward_hook(get_features('layer1'))
model.layer2.register_forward_hook(get_features('layer2'))
model.layer3.register_forward_hook(get_features('layer3'))
model.layer4.register_forward_hook(get_features('layer4'))

# Process your image as needed here before passing it to the model
# Assuming 'processed_image' is your input tensor ready to be passed through the model

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)
# Access the outputs
layer1_output = outputs['layer1']
layer2_output = outputs['layer2']
layer3_output = outputs['layer3']
layer4_output = outputs['layer4']

print("Shape of layer1 output: ", layer1_output.shape)
print("Shape of layer2 output: ", layer2_output.shape)
print("Shape of layer3 output: ", layer3_output.shape)
print("Shape of layer4 output: ", layer4_output.shape)

# Now you can use layer1_output, layer2_output, etc., for your tasks


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


<bound method Module.named_modules of ResNetModel(
  (embedder): ResNetEmbeddings(
    (embedder): ResNetConvLayer(
      (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (encoder): ResNetEncoder(
    (stages): ModuleList(
      (0): ResNetStage(
        (layers): Sequential(
          (0): ResNetBottleNeckLayer(
            (shortcut): ResNetShortCut(
              (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (layer): Sequential(
              (0): ResNetConvLayer(
                (convolution): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bia

AttributeError: 'ResNetModel' object has no attribute 'layer2'

In [59]:
from transformers import ResNetConfig, ResNetModel, AutoImageProcessor
from datasets import load_dataset
import torch

# Load the dataset
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

# Initialize the image processor
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")

# Process the image
inputs = image_processor(image, return_tensors="pt")

# Print input shapes
print("Inputs shape :", inputs.pixel_values.shape)

# Create a configuration that specifies desired output features
config = ResNetConfig.from_pretrained("microsoft/resnet-50", out_features=['stem'], output_hidden_states=True)

print("Config asd: ", config.hidden_sizes[0])

print("Features to extract:", config.out_features)

# Load the model with the specified configuration
model = ResNetModel.from_pretrained("microsoft/resnet-50", config=config)

print("Model config:", model.config.output_hidden_states)

# Forward pass (disable gradient computation)
with torch.no_grad():
    outputs = model(**inputs)
    
import matplotlib.pyplot as plt
import numpy as np

# Assuming `outputs` is the result of the forward pass with `output_hidden_states=True`
# and you have a batch size of 1.

def plot_all_feature_maps(feature_maps, max_feature_maps_per_stage=16):
    """
    Plots a subset of feature maps across all stages.
    
    Args:
    - feature_maps (list of torch.Tensor): A list where each element is a tensor
      representing the output from one stage of the model.
    - max_feature_maps_per_stage (int): Maximum number of feature maps to display per stage.
    """
    for stage_idx, stage_output in enumerate(feature_maps):
        num_feature_maps = stage_output.size(1)  # Number of feature maps in this stage
        num_plots = min(num_feature_maps, max_feature_maps_per_stage)  # Limit the number of feature maps
        
        # Set up subplot grid
        num_columns = 4  # You can adjust this to fit your screen better
        num_rows = int(np.ceil(num_plots / num_columns))
        fig, axes = plt.subplots(num_rows, num_columns, figsize=(2*num_columns, 2*num_rows))
        
        if num_rows == 1 and num_columns == 1:
            axes = np.array([[axes]])
        elif num_rows == 1 or num_columns == 1:
            axes = np.expand_dims(axes, 0 if num_rows > 1 else 1)

        for i in range(num_plots):
            row, col = divmod(i, num_columns)
            feature_map = stage_output[0, i].cpu().numpy()  # Select the i-th feature map
            axes[row, col].imshow(feature_map, cmap='viridis')
            axes[row, col].set_title(f'Stage {stage_idx+1} Map {i+1}')
            axes[row, col].axis('off')
        
        # Hide any unused subplots
        for j in range(i+1, num_rows*num_columns):
            row, col = divmod(j, num_columns)
            axes[row, col].axis('off')

        plt.tight_layout()
        plt.show()

# Now, visualize all feature maps from all stages
feature_maps = outputs.hidden_states
print("Number of stages:", len(feature_maps))
print("Number of feature maps per stage:", [stage_output.size(1) for stage_output in feature_maps])

#plot_all_feature_maps(feature_maps, max_feature_maps_per_stage=64)



Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Inputs shape : torch.Size([1, 3, 224, 224])
Config asd:  256
Features to extract: ['stem']
Model config: True
Number of stages: 5
Number of feature maps per stage: [64, 256, 512, 1024, 2048]
